# Tensormesh Smoke Test Notebook

This notebook performs a minimal request against a Tensormesh-deployed model through an OpenAI-compatible endpoint.

## Prerequisites

- Create `.env` from `.env.example`.
- Set `OPENAI_BASE_URL`, `OPENAI_API_KEY`, `TENSORMESH_USER_ID`, and `OPENAI_MODEL`.
- Install dependency if needed: `pip install openai`.

## Workflow

1. Load values from `.env`.
2. Validate required environment variables.
3. Send one chat completion request.
4. Print response text, metadata, and raw JSON payload.

In [1]:
from pathlib import Path
import os


def load_dotenv(dotenv_path: Path = Path('.env')) -> None:
    if not dotenv_path.exists():
        return
    for raw_line in dotenv_path.read_text(encoding='utf-8').splitlines():
        line = raw_line.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue
        key, value = line.split('=', 1)
        key = key.strip()
        value = value.strip().strip('\"').strip("'")
        os.environ.setdefault(key, value)


def require_env(name: str) -> str:
    value = os.environ.get(name)
    if not value:
        raise RuntimeError(f'Missing required env var: {name}')
    return value


def mask_secret(secret: str, keep: int = 4) -> str:
    if len(secret) <= keep:
        return '*' * len(secret)
    return '*' * (len(secret) - keep) + secret[-keep:]


load_dotenv()

In [2]:
base_url = require_env('OPENAI_BASE_URL').rstrip('/')
api_key = require_env('OPENAI_API_KEY')
user_id = require_env('TENSORMESH_USER_ID')
model = require_env('OPENAI_MODEL')

if not base_url.endswith('/v1'):
    print('Note: OPENAI_BASE_URL usually ends with /v1')

print('Configuration')
print(f'  OPENAI_BASE_URL   : {base_url}')
print(f'  OPENAI_MODEL      : {model}')
print(f'  TENSORMESH_USER_ID: {user_id}')
print(f'  OPENAI_API_KEY    : {mask_secret(api_key)}')

Configuration
  OPENAI_BASE_URL   : https://external.nebius.tensormesh.ai/v1
  OPENAI_MODEL      : openai/gpt-oss-20b
  TENSORMESH_USER_ID: 60b01a41-c8a4-4234-80a5-ef18309b560c
  OPENAI_API_KEY    : *******************************fdMX


In [3]:
try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError(
        'Failed to import openai package. Install with: pip install openai'
    ) from e

In [4]:
import time

client = OpenAI(
    api_key=api_key,
    base_url=base_url,
    default_headers={'X-User-Id': user_id},
)

request_messages = [
    {
        'role': 'system',
        'content': 'You are a helpful assistant.'
    },
    {
        'role': 'user',
        'content': 'Write a short haiku about cloud compute.'
    },
]

t0 = time.perf_counter()
response = client.chat.completions.create(
    model=model,
    messages=request_messages,
    temperature=0.7,
)
latency_ms = (time.perf_counter() - t0) * 1000
print(f'Smoke test request completed in {latency_ms:.1f} ms')

Smoke test request completed in 3051.9 ms


In [5]:
assistant_text = response.choices[0].message.content or ''
print('Assistant response')
print('=' * 80)
print(assistant_text)

Assistant response
Sky of bits hums bright  
Virtual cores hum in silence  
Data flows like mist


In [6]:
usage = getattr(response, 'usage', None)
print('Response summary')
print(f"  id    : {getattr(response, 'id', None)}")
print(f"  model : {getattr(response, 'model', None)}")
print(f"  usage : {usage}")

Response summary
  id    : chatcmpl-1e1ffc55-2f0b-4cc3-83b7-5f480684e3ed
  model : openai/gpt-oss-20b
  usage : CompletionUsage(completion_tokens=521, prompt_tokens=88, total_tokens=609, completion_tokens_details=None, prompt_tokens_details=None)


In [7]:
import json

raw = response.model_dump() if hasattr(response,
                                       'model_dump') else dict(response)
print('Raw response payload')
print('=' * 80)
print(json.dumps(raw, indent=2, ensure_ascii=False))

Raw response payload
{
  "id": "chatcmpl-1e1ffc55-2f0b-4cc3-83b7-5f480684e3ed",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Sky of bits hums bright  \nVirtual cores hum in silence  \nData flows like mist",
        "refusal": null,
        "role": "assistant",
        "annotations": null,
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning": "We need a haiku 5-7-5 syllables. Short haiku about cloud compute. Let's craft: \"Sky of bits hums, / virtual cores in silence, / data flows like mist.\" Count syllables: Sky(1) of(1) bits(1) hums(1) =4? Actually \"Sky of bits hums\" = 1+1+1+1 =4. Need 5. Maybe \"Sky of bits hums bright\". That's 5: Sky(1) of(1) bits(1) hums(1) bright(1) =5. Next line: \"virtual cores in silence\" count: virt(1) ual(1) cores(1) in(1) si(1) lence(1) =6? Let's count: virt (1), ual (1) =2, cores (1)=3, in(1)=4, si(1)=5, lence(1)=6. So 